# TP Big Data - Analyse Batch des Données TAN avec Spark

Ce notebook permet d'analyser en mode batch les données collectées depuis l'API TAN et stockées dans Kafka. Nous effectuerons deux analyses distinctes :

1. Distribution des arrêts de transport et leurs caractéristiques
2. Analyse des temps d'attente par ligne et par arrêt

## Initialisation de Spark

Commençons par configurer notre session Spark.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, explode, regexp_extract, count, avg, min, max
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Création de la session Spark
spark = SparkSession.builder \
    .appName("TAN Batch Analysis") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \
    .getOrCreate()

# Réduire les messages de log
spark.sparkContext.setLogLevel("WARN")

print("Session Spark initialisée!")

/opt/conda/lib/python3.11/site-packages/pyspark/bin/spark-class: line 71: /usr/lib/jvm/java-11-openjdk-amd64/bin/java: No such file or directory
/opt/conda/lib/python3.11/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript


RuntimeError: Java gateway process exited before sending its port number

## Analyse 1 : Distribution des arrêts de transport

Nous allons analyser la distribution des arrêts de bus/tram, leur distance et les lignes qu'ils desservent.

### Définition du schéma et lecture des données

In [ ]:
# Définition du schéma pour les données d'arrêts
stop_schema = StructType([
    StructField("stop_code", StringType(), True),
    StructField("stop_name", StringType(), True),
    StructField("stop_distance", StringType(), True),
    StructField("ligne", ArrayType(StructType([
        StructField("numLigne", StringType(), True)
    ])), True),
    StructField("timestamp", StringType(), True)
])

# Lecture des données depuis Kafka en mode batch
stops_df = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:9092") \
    .option("subscribe", "tan_stops") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load() \
    .selectExpr("CAST(value AS STRING) AS json")

# Parsing du JSON
parsed_stops = stops_df \
    .select(from_json("json", stop_schema).alias("data")) \
    .select("data.*")

# Affichage d'un échantillon des données
parsed_stops.show(5, truncate=False)

### Traitement des données

Extrayons maintenant les informations pertinentes et convertissons la distance en valeur numérique.

In [ ]:
# Extraction de la partie numérique de la distance (ex: "256 m" -> 256)
parsed_stops = parsed_stops \
    .withColumn("distance_meters", 
                regexp_extract("stop_distance", r"(\d+)", 1).cast(FloatType()))

# Extraction des numéros de ligne dans une colonne distincte
stops_with_lines = parsed_stops \
    .withColumn("line_numbers", explode("ligne")) \
    .select("stop_code", "stop_name", "distance_meters", "line_numbers.numLigne")

# Mise en cache pour optimiser les accès multiples
stops_with_lines.cache()

# Affichage des données après traitement
stops_with_lines.show(5)

### Analyse et visualisation 1: Distribution des arrêts par distance

In [ ]:
# Conversion en DataFrame Pandas pour visualisation
stops_pandas = stops_with_lines.toPandas()

# Statistiques descriptives
stats = stops_pandas["distance_meters"].describe()
print("Statistiques descriptives des distances:")
print(stats)

# Visualisation: Distribution des arrêts par distance
plt.figure(figsize=(10, 6))
sns.histplot(data=stops_pandas, x="distance_meters", bins=20, kde=True)
plt.title("Distribution des arrêts par distance")
plt.xlabel("Distance (mètres)")
plt.ylabel("Nombre d'arrêts")
plt.axvline(x=stops_pandas["distance_meters"].mean(), color='r', linestyle='--', 
            label=f'Moyenne: {stops_pandas["distance_meters"].mean():.1f}m')
plt.legend()
plt.show()

### Analyse et visualisation 2: Nombre d'arrêts par ligne

In [ ]:
# Comptage des arrêts par ligne
line_counts = stops_pandas["numLigne"].value_counts().reset_index()
line_counts.columns = ['Ligne', 'Nombre_arrets']
line_counts = line_counts.sort_values(by='Nombre_arrets', ascending=False).head(10)

# Visualisation: Top 10 des lignes par nombre d'arrêts
plt.figure(figsize=(12, 6))
sns.barplot(data=line_counts, x='Ligne', y='Nombre_arrets')
plt.title("Top 10 des lignes par nombre d'arrêts desservis")
plt.xlabel("Ligne")
plt.ylabel("Nombre d'arrêts")
plt.xticks(rotation=45)
plt.show()

## Analyse 2 : Temps d'attente aux arrêts

Analysons maintenant les temps d'attente aux différents arrêts et pour différentes lignes.

### Définition du schéma et lecture des données

In [ ]:
# Définition du schéma pour les données de temps d'attente
wait_schema = StructType([
    StructField("ligne", StructType([
        StructField("numLigne", StringType(), True)
    ]), True),
    StructField("terminus", StringType(), True),
    StructField("arret", StructType([
        StructField("codeArret", StringType(), True),
        StructField("libelle", StringType(), True)
    ]), True),
    StructField("temps", StringType(), True),
    StructField("timestamp", StringType(), True)
])

# Lecture des données depuis Kafka en mode batch
wait_df = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:9092") \
    .option("subscribe", "tan_wait_times") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load() \
    .selectExpr("CAST(value AS STRING) AS json")

# Parsing du JSON
parsed_wait = wait_df \
    .select(from_json("json", wait_schema).alias("data")) \
    .select("data.*")

# Affichage d'un échantillon des données
parsed_wait.show(5, truncate=False)

### Traitement des données

In [ ]:
# Aplatissement de la structure imbriquée
flattened_wait = parsed_wait \
    .select(
        col("ligne.numLigne").alias("line_number"),
        col("terminus").alias("destination"),
        col("arret.codeArret").alias("stop_code"),
        col("arret.libelle").alias("stop_name"),
        col("temps").alias("wait_time"),
        col("timestamp")
    )

# Extraction de la valeur numérique du temps d'attente (ex: "5 mn" -> 5)
flattened_wait = flattened_wait \
    .withColumn("wait_minutes", 
                regexp_extract("wait_time", r"(\d+)", 1).cast(FloatType()))

# Mise en cache pour optimiser les accès multiples
flattened_wait.cache()

# Affichage des données après traitement
flattened_wait.show(5)

### Analyse et visualisation 1: Temps d'attente moyen par ligne

In [ ]:
# Calcul du temps d'attente moyen par ligne
wait_by_line = flattened_wait.groupBy("line_number") \
    .agg(
        avg("wait_minutes").alias("avg_wait"),
        count("*").alias("count")
    ) \
    .filter(col("count") > 3)  # Au moins 3 observations par ligne
    
# Conversion en DataFrame Pandas
wait_by_line_pandas = wait_by_line.toPandas()

# Tri par temps d'attente moyen décroissant
wait_by_line_pandas = wait_by_line_pandas.sort_values(by="avg_wait", ascending=False)

# Visualisation: Temps d'attente moyen par ligne
plt.figure(figsize=(12, 6))
sns.barplot(data=wait_by_line_pandas.head(10), x="line_number", y="avg_wait")
plt.title("Temps d'attente moyen par ligne")
plt.xlabel("Ligne")
plt.ylabel("Temps d'attente moyen (minutes)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Analyse et visualisation 2: Distribution des temps d'attente

In [ ]:
# Conversion en DataFrame Pandas pour visualisation
wait_pandas = flattened_wait.toPandas()

# Statistiques descriptives
wait_stats = wait_pandas["wait_minutes"].describe()
print("Statistiques descriptives des temps d'attente:")
print(wait_stats)

# Visualisation: Distribution des temps d'attente
plt.figure(figsize=(10, 6))
sns.histplot(data=wait_pandas, x="wait_minutes", bins=15, kde=True)
plt.title("Distribution des temps d'attente")
plt.xlabel("Temps d'attente (minutes)")
plt.ylabel("Fréquence")
plt.axvline(x=wait_pandas["wait_minutes"].mean(), color='r', linestyle='--', 
            label=f'Moyenne: {wait_pandas["wait_minutes"].mean():.1f} min')
plt.legend()
plt.show()

### Analyse et visualisation 3: Temps d'attente par arrêt

In [ ]:
# Calcul du temps d'attente moyen par arrêt
wait_by_stop = flattened_wait.groupBy("stop_name") \
    .agg(
        avg("wait_minutes").alias("avg_wait"),
        count("*").alias("count")
    ) \
    .filter(col("count") > 3)  # Au moins 3 observations par arrêt
    
# Conversion en DataFrame Pandas
wait_by_stop_pandas = wait_by_stop.toPandas()

# Tri par temps d'attente moyen décroissant
wait_by_stop_pandas = wait_by_stop_pandas.sort_values(by="avg_wait", ascending=False)

# Visualisation: Top 10 des arrêts avec le temps d'attente le plus long
plt.figure(figsize=(14, 7))
sns.barplot(data=wait_by_stop_pandas.head(10), x="stop_name", y="avg_wait")
plt.title("Top 10 des arrêts avec le temps d'attente le plus long")
plt.xlabel("Arrêt")
plt.ylabel("Temps d'attente moyen (minutes)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Synthèse des résultats et insights

Récapitulons les principales découvertes de notre analyse batch.

In [ ]:
# Statistiques générales
print("=== Statistiques sur les arrêts ===")
print(f"Nombre total d'arrêts uniques: {stops_pandas['stop_name'].nunique()}")
print(f"Distance moyenne des arrêts: {stops_pandas['distance_meters'].mean():.2f} mètres")
print(f"Distance médiane: {stops_pandas['distance_meters'].median():.2f} mètres")
print(f"Distance minimale: {stops_pandas['distance_meters'].min():.2f} mètres")
print(f"Distance maximale: {stops_pandas['distance_meters'].max():.2f} mètres")

print("\n=== Statistiques sur les temps d'attente ===")
print(f"Temps d'attente moyen: {wait_pandas['wait_minutes'].mean():.2f} minutes")
print(f"Temps d'attente médian: {wait_pandas['wait_minutes'].median():.2f} minutes")
print(f"Temps d'attente minimal: {wait_pandas['wait_minutes'].min():.2f} minutes")
print(f"Temps d'attente maximal: {wait_pandas['wait_minutes'].max():.2f} minutes")

print("\n=== Top 3 des lignes les plus fréquentes ===")
for i, row in line_counts.head(3).iterrows():
    print(f"Ligne {row['Ligne']}: {row['Nombre_arrets']} arrêts")

print("\n=== Top 3 des lignes avec le temps d'attente le plus long ===")
for i, row in wait_by_line_pandas.head(3).iterrows():
    print(f"Ligne {row['line_number']}: {row['avg_wait']:.2f} minutes en moyenne")

print("\n=== Top 3 des arrêts avec le temps d'attente le plus long ===")
for i, row in wait_by_stop_pandas.head(3).iterrows():
    print(f"{row['stop_name']}: {row['avg_wait']:.2f} minutes en moyenne")

## Conclusion

Cette analyse batch nous a permis de mieux comprendre la distribution spatiale des arrêts de transport en commun à Nantes ainsi que les tendances en termes de temps d'attente. Ces informations sont précieuses pour :

1. Identifier les zones de la ville bien ou mal desservies par les transports en commun
2. Repérer les lignes et arrêts qui pourraient bénéficier d'une amélioration de la fréquence
3. Aider les voyageurs à mieux planifier leurs déplacements en fonction des temps d'attente moyens

Dans le notebook suivant, nous explorerons ces données en streaming pour obtenir des analyses en temps réel.

In [ ]:
# Libérer les ressources
stops_with_lines.unpersist()
flattened_wait.unpersist()
spark.stop()